In [24]:
import os
import os.path
import sys
import pandas as pd
import numpy as np

In [25]:
LOCAL_PATH = 'data'
RAW_TRACE = 'ashakan_raw.pcapng'
TRACE_FEATURE_FILE = 'tcp_flow_features.csv'
BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

In [26]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [27]:
import subprocess
if not os.path.exists(os.path.join(LOCAL_PATH, TRACE_FEATURE_FILE)):
    tshark_command = subprocess.Popen('tshark -r {} -Y tcp -T fields -e ip.src -e ip.dst -e tcp.srcport -e tcp.dstport -e tcp.len -e frame.time_relative -e tcp.seq -e tcp.ack -e tcp.flags.ack -e tcp.flags.syn -e tcp.flags.fin -e tcp.stream -Eheader=y -Eseparator=, > {}'.format(LOCAL_PATH + "/" + RAW_TRACE, LOCAL_PATH + "/" + TRACE_FEATURE_FILE), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out_data, err_data = tshark_command.communicate()
    out_data, err_data = out_data.decode('utf-8'), err_data.decode('utf-8')
    if err_data != '':
        pass
#         print(err_data)

In [29]:
trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TRACE_FEATURE_FILE))
trace_df['src_addr'] = trace_df['ip.src'] + ":" + trace_df['tcp.srcport'].apply(str)
trace_df['dst_addr'] = trace_df['ip.dst'] + ":" + trace_df['tcp.dstport'].apply(str)

In [30]:
trace_df.shape

(3275767, 14)

In [31]:
trace_df.dtypes

ip.src                  object
ip.dst                  object
tcp.srcport              int64
tcp.dstport              int64
tcp.len                  int64
frame.time_relative    float64
tcp.seq                  int64
tcp.ack                  int64
tcp.flags.ack            int64
tcp.flags.syn            int64
tcp.flags.fin            int64
tcp.stream               int64
src_addr                object
dst_addr                object
dtype: object

In [32]:
trace_df.describe()

,tcp.srcport,tcp.dstport,tcp.len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream
count,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06
mean,1.959040e+04,1.189782e+04,1.001266e+03,2.770661e+01,3.896565e+08,5.583203e+07,9.967156e-01,6.052323e-03,5.241521e-03,1.130049e+03
std,2.130404e+04,1.709713e+04,6.454179e+02,1.189198e+01,4.620892e+08,1.584937e+08,5.721569e-02,7.756091e-02,7.220837e-02,3.101892e+03
min,2.100000e+01,2.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.450000e+02,2.770000e+03,1.180000e+02,1.734808e+01,8.033140e+05,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.770000e+03,2.901000e+03,1.460000e+03,2.784759e+01,2.364588e+08,3.034090e+05,1.000000e+00,0.000000e+00,0.000000e+00,1.400000e+01
75%,4.092000e+04,9.876000e+03,1.460000e+03,3.792557e+01,6.421602e+08,1.348768e+06,1.000000e+00,0.000000e+00,0.000000e+00,3.800000e+01
max,6.553400e+04,6.551600e+04,1.460000e+03,4.828968e+01,4.294967e+09,4.294967e+09,1.000000e+00,1.000000e+00,1.000000e+00,1.798300e+04


In [33]:
trace_df.head()

,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream,src_addr,dst_addr
0,10.2.21.28,10.2.4.172,389,63633,293,0.000000,1,1,1,0,0,0,10.2.21.28:389,10.2.4.172:63633
1,10.2.4.179,10.2.4.146,40920,9876,1460,0.000001,1,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876
2,10.2.4.179,10.2.4.146,40920,9876,1460,0.000002,1461,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876
3,10.2.4.179,10.2.4.146,40920,9876,1460,0.000002,2921,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876
4,10.2.4.179,10.2.4.146,40920,9876,1460,0.000002,4381,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876


In [34]:
# run 'jupyter nbextension enable --py --sys-prefix widgetsnbextension' first
from tqdm import tqdm
def to_feature_df(raw_trace_df,sampling_rate=1.0,upsampled=False):
    def calculate_two_way_tcp(df):
        def get_statistical_features(df, criter, feature_name,name_pred):
            # upsampling
            feature_avg = df[criter][feature_name].mean()
            feature_avg = -1 if pd.isnull(feature_avg) else feature_avg / sampling_rate if upsampled else feature_avg
            feature_min = df[criter][feature_name].min()
            feature_min = -1 if pd.isnull(feature_min) else feature_min
            feature_max = df[criter][feature_name].max()
            feature_max = -1 if pd.isnull(feature_max) else feature_max
            feature_std = df[criter][feature_name].std()
            feature_std = -1 if pd.isnull(feature_std) else feature_std
            feature_sum = df[criter][feature_name].sum()
            feature_sum = -1 if pd.isnull(feature_sum) else feature_sum / sampling_rate if upsampled else feature_sum
            feature_count = df[criter][feature_name].count()
            feature_count = feature_count / sampling_rate if upsampled else feature_count
            return {'avg('+name_pred+')':feature_avg,'std('+name_pred+')':feature_std,'min('+name_pred+')':feature_min,'max('+name_pred+')':feature_max,'count('+name_pred[0:8]+')':feature_count, 'sum('+name_pred+')':feature_sum}
        
        addrs = list(set(np.append(df['src_addr'].unique(), df['dst_addr'].unique())))
        if len(addrs) != 2:
            raise
        stat = get_statistical_features(df, df['src_addr'] == addrs[0],'tcp.len','forw_pkt_len')
        stat.update(get_statistical_features(df, df['src_addr'] == addrs[1],'tcp.len','back_pkt_len'))
        return pd.Series(stat)

    trace_df = raw_trace_df
    tcp_flow_df = pd.DataFrame()
    # upsampling
    tcp_flow_df['avg(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].mean()
    tcp_flow_df['stddev(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].std().fillna(-1)
    tcp_flow_df['min(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].min()
    tcp_flow_df['max(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].max()
    tcp_flow_df['tot_pkt'] = trace_df.groupby('tcp.stream')['tcp.len'].count()
    tcp_flow_df['tot_byte'] = trace_df.groupby('tcp.stream')['tcp.len'].sum()
    tcp_flow_df['rel_start'] = trace_df.groupby('tcp.stream')['frame.time_relative'].min()
    tcp_flow_df['duration'] = trace_df.groupby('tcp.stream')['frame.time_relative'].max() - tcp_flow_df['rel_start']
    if not upsampled:
        tqdm.pandas(desc='{} samp rate no upsampling'.format(sampling_rate))
    else:
        tcp_flow_df['avg(tcp_pkt_len)'] /= sampling_rate
        tcp_flow_df['tot_pkt'] /= sampling_rate
        tcp_flow_df['tot_byte'] /= sampling_rate
        tqdm.pandas(desc='{} samp rate with upsampling'.format(sampling_rate))
    two_way_flow_df = trace_df.groupby('tcp.stream')[['tcp.len','src_addr','dst_addr']].progress_apply(calculate_two_way_tcp)
    tcp_flow_df = pd.concat([tcp_flow_df,two_way_flow_df],axis=1)
    return tcp_flow_df

def sample_trace(raw_trace_df,sampling_rate):
    import time
    return raw_trace_df.sample(frac=sampling_rate, random_state=int(time.time()))

In [35]:
SAMPLING_RATES = [5,10,15,20,40,60,80,100]

In [36]:
# run 'jupyter nbextension enable --py --sys-prefix widgetsnbextension' first
from tqdm import tqdm_notebook
# packet-based random sampling
for sampling_percent in tqdm_notebook(SAMPLING_RATES,desc='Sampling'):
    sampling_rate = sampling_percent / 100.0
    sampled_df = sample_trace(trace_df, sampling_rate)
    if sampling_percent < 100:
        to_feature_df(sampled_df, sampling_rate, upsampled=False).to_csv(os.path.join(LOCAL_PATH,'packet_rand_{PERCENT}%_no_upsampling.csv'.format(PERCENT=sampling_percent)))
        to_feature_df(sampled_df, sampling_rate, upsampled=True).to_csv(os.path.join(LOCAL_PATH,'packet_rand_{PERCENT}%_with_upsampling.csv'.format(PERCENT=sampling_percent)))
    elif sampling_percent == 100:
        to_feature_df(sampled_df, sampling_rate, upsampled=False).to_csv('{PATH}/packet_full.csv'.format(PATH=LOCAL_PATH))
    else:
        raise

HBox(children=(IntProgress(value=0, description='Sampling', max=8), HTML(value='')))

0.05 samp rate with upsampling:  13%|█▎        | 1014/7874 [00:09<01:09, 99.04it/s] 

KeyboardInterrupt: 

0.05 samp rate with upsampling:  13%|█▎        | 1022/7874 [00:20<01:09, 99.04it/s]